In [ ]:
import globus_automate_client
import mdf_toolbox
import json

In [ ]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [ ]:
# Required secret keys for testing the Curation Flow
# Not required for deployment, or for testing of the Transfer Flow
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

In [ ]:
# Load Transfer Loop Flow definition from JSON
with open("transfer_loop_flow_def.json") as f:
    transfer_flow_def = json.load(f)

In [ ]:
transfer_deploy_res = flows_client.deploy_flow(
    flow_definition=transfer_flow_def["definition"],
    title=transfer_flow_def["title"],
    description=transfer_flow_def["description"],
    visible_to=transfer_flow_def["visible_to"],
    runnable_by=transfer_flow_def["runnable_by"],
    administered_by=transfer_flow_def["administered_by"],
    # TODO: Make JSONSchema
    input_schema={},
    validate_definition=True,
    validate_input_schema=True
)
transfer_flow_id = transfer_deploy_res["id"]
transfer_flow_scope = transfer_deploy_res["globus_auth_scope"]

# Save Flow ID/scope for future use
with open("transfer_loop_flow_info.json", 'w') as f:
    transfer_flow_info = {
        "flow_id": transfer_flow_id,
        "flow_scope": transfer_flow_scope
    }
    json.dump(transfer_flow_info, f, indent=4)

In [ ]:
# Alternatively, to keep the same Flow ID and scope, update the Flow
# Deploying recommended over updating - Automate Flow updates are not stable
with open("transfer_loop_flow_info.json") as f:
    transfer_flow_info = json.load(f)
    transfer_flow_id = transfer_flow_info["flow_id"]
    transfer_flow_scope = transfer_flow_info["flow_scope"]

transfer_update_res = flows_client.update_flow(
    transfer_flow_id,
    flow_definition=transfer_flow_def["definition"],
    title=transfer_flow_def["title"],
    description=transfer_flow_def["description"],
    visible_to=transfer_flow_def["visible_to"],
    runnable_by=transfer_flow_def["runnable_by"],
    administered_by=transfer_flow_def["administered_by"],
    # TODO
    input_schema={},
    validate_definition=True,
    validate_input_schema=True
)

In [ ]:
# Print the ID and scope, which is useful for debugging purposes.
print(transfer_flow_id)
print(transfer_flow_scope)

In [ ]:
# Get test input for Flow
# NOTE: You may need to change the test input if you don't have permissions on all testing endpoints
with open("transfer_loop_flow_input.json") as f:
    transfer_flow_input = json.load(f)

In [ ]:
# Initiate Flow as you
# Careful to run this cell only once per test - the first Flow's Action ID will be overwritten and lost
# if this is run a second time.
transfer_flow_res = flows_client.run_flow(transfer_flow_id, transfer_flow_scope, transfer_flow_input)
transfer_flow_res.data

In [ ]:
# Check status of Flow
# This cell should be run multiple times, to check the status until the Flow succeeds or fails
transfer_status = flows_client.flow_action_status(transfer_flow_id, transfer_flow_scope, transfer_flow_res["action_id"]).data
print(json.dumps(transfer_status, indent=4, sort_keys=True))

In [ ]:
# Check the Flow Log
# The Log is very verbose and only necessary when debugging errors in Flow execution
flows_client.flow_action_log(transfer_flow_id, transfer_flow_scope, transfer_flow_res["action_id"], limit=100).data

In [ ]:
# Load Curation Flow from JSON
with open("curation_flow_def.json") as f:
    curation_flow_def = json.load(f)

In [ ]:
# Load other configuration variables
# Please set these in the configuration file, not in-line here
with open("curation_flow_config.json") as f:
    curation_config = json.load(f)

# Default templates for emails
curation_flow_def["definition"]["States"]["SendCuratorEmails"]["Parameters"]["body_template.="] = \
        ("`$.curator_template` or '" + curation_config["curator_email_template"] + "'")
curation_flow_def["definition"]["States"]["SendAuthorEmail"]["Parameters"]["body_template.="] = \
        ("`$.author_template` or '" + curation_config["author_email_template"] + "'")

# Curation text box prompt and default preamble
curation_flow_def["definition"]["States"]["CurateData"]["Parameters"]["landing_page"]["text_input_form_prompt"] = \
        curation_config["text_box_prompt"]
curation_flow_def["definition"]["States"]["CurateData"]["Parameters"]["landing_page"]["preamble_text.="] = \
        ("`$.curation_text` or '" + curation_config["preamble_text"] + "'")

In [ ]:
curation_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=curation_flow_def["definition"],
    title=curation_flow_def["title"],
    description=curation_flow_def["description"],
    visible_to=curation_flow_def["visible_to"],
    runnable_by=curation_flow_def["runnable_by"],
    administered_by=curation_flow_def["administered_by"],
    # TODO: Make rough schema outline into JSONSchema
    input_schema={}, #  curation_flow_def["schema"],
    validate_definition=True,
    validate_input_schema=True
)
curation_flow_id = curation_flow_deploy_res["id"]
curation_flow_scope = curation_flow_deploy_res["globus_auth_scope"]

# Save Flow ID/scope for future use
with open("curation_flow_info.json", 'w') as f:
    curation_flow_info = {
        "flow_id": curation_flow_id,
        "flow_scope": curation_flow_scope
    }
    json.dump(curation_flow_info, f, indent=4)

In [ ]:
print(curation_flow_id)
print(curation_flow_deploy_res["action_url"])
print(curation_flow_scope)

In [ ]:
# Get test input
with open("curation_flow_input.json") as f:
    curation_flow_input = json.load(f)
    
# Add private credentials
curation_flow_input["_private_send_credentials"] = [{
    "credential_type": "smtp",
    "credential_value": {
        "hostname": smtp_hostname,
        "username": smtp_user,
        "password": smtp_pass

    }
}]

In [ ]:
# Initiate Flow as you
# Careful to run this cell only once per test - the first Flow's Action ID will be overwritten and lost
# if this is run a second time.
curation_flow_res = flows_client.run_flow(curation_flow_id, curation_flow_scope, curation_flow_input)
curation_flow_res.data

In [ ]:
# Check status of Flow
# This cell should be run multiple times, to check the status until the Flow succeeds or fails
curation_status = flows_client.flow_action_status(curation_flow_id, curation_flow_scope, curation_flow_res["action_id"]).data
print(json.dumps(curation_status, indent=4, sort_keys=True))

In [ ]:
# Check the Flow Log
# The Log is very verbose and only necessary when debugging errors in Flow execution
flows_client.flow_action_log(curation_flow_id, curation_flow_scope, curation_flow_res["action_id"], limit=100).data